In [1]:
# -*- coding: UTF-8 -*-
import tensorflow as tf
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from matplotlib import pyplot as plt
from Preprocess import label_for_nn

In [2]:
benign_path = '/home/user01/Desktop/DDSM/4_benign_Pleo_10_ROI_subpic'#input('良性子图路径')#
cancer_path = '/home/user01/Desktop/DDSM/4_cancer_Pleo_10_ROI_subpic'#input('恶性子图路径')#
saver_path = '/home/user01/Desktop/DDSM/model/VGG11'#input('模型保存路径')#

In [3]:
size = 128
train_time = 100
batch_size = 10
learning_rate = 1e-5
dropout_rate = 0.5

In [4]:
def get_trainsets_and_testsets():
    (train_x ,train_y),(test_x,test_y) = label_for_nn(benign_path,cancer_path,scale=0.2,kind='RGB')
    train_x = train_x.reshape(train_x.shape[0],size,size,3).astype('float32')/255.0
    test_x = test_x.reshape(test_x.shape[0],size,size,3).astype('float32')/255.0
    return train_x , test_x ,train_y ,test_y

In [5]:
def Weight_variable(shape,name):
    initial = tf.truncated_normal(shape,stddev=0.027)
    return tf.Variable(initial,name=name)

In [6]:
def biases_variable(shape,name):
    initial = tf.constant(0.0,shape=shape)
    return tf.Variable(initial,name=name)

In [7]:
def conv2d_P(x,W,s):
    return tf.nn.conv2d(x,W,strides=[1,s,s,1],padding='SAME')

In [8]:
def conv2d(x,W,s):
    return tf.nn.conv2d(x,W,strides=[1,s,s,1],padding='VALID')

In [9]:
def max_pooling2x2(x):
    return tf.nn.max_pool(x,ksize=[1,3,3,1],strides=[1,2,2,1],padding='VALID')

In [10]:
def cnn_train():
    with tf.name_scope('inputs'):
        x = tf.placeholder(tf.float32, [None, size, size, 3])
        y = tf.placeholder(tf.float32, [None, 2])
        keep_prob = tf.placeholder(tf.float32)
    #conv1#
    with tf.name_scope('layer_1'):
        with tf.name_scope('weight_1'):
            w_conv1 = Weight_variable([3,3,3,64],'w_1')
        with tf.name_scope('biases_1'):
            b_conv1 = biases_variable([64],'b_1')
        with tf.name_scope('conv_1'):
            conv1 = tf.nn.relu(conv2d_P(x,w_conv1,1)+b_conv1)
        with tf.name_scope('pool_1'):
            pool1 = max_pooling2x2(conv1)
    #conv2#
    with tf.name_scope('layer_2'):
        with tf.name_scope('weight_2'):
            w_conv2 = Weight_variable([3,3,64,128],'w_2')
        with tf.name_scope('biases_2'):
            b_conv2 = biases_variable([128],'b_2')
        with tf.name_scope('conv_2'):
            conv2 = tf.nn.relu(conv2d_P(pool1,w_conv2,1)+b_conv2)
        with tf.name_scope('pool_2'):
            pool2 = max_pooling2x2(conv2)

    #conv3#
    with tf.name_scope('layer_3'):
        with tf.name_scope('weight_3'):
            w_conv3 = Weight_variable([3,3,128,256],'w_3')
        with tf.name_scope('biases_3'):
            b_conv3 = biases_variable([256],'b_3')
        with tf.name_scope('conv_3'):
            conv3 = tf.nn.relu(conv2d_P(pool2,w_conv3,1)+b_conv3)

    #conv4#
    with tf.name_scope('layer_4'):
        with tf.name_scope('weight_4'):
            w_conv4 = Weight_variable([3,3,256,256],'w_4')
        with tf.name_scope('biases_4'):
            b_conv4 = biases_variable([256],'b_4')
        with tf.name_scope('conv_4'):
            conv4 = tf.nn.relu(conv2d_P(conv3,w_conv4,1)+b_conv4)
        with tf.name_scope('pool_3'):
            pool3 = max_pooling2x2(conv4)

    #conv5#
    with tf.name_scope('layer_5'):
        with tf.name_scope('weight_5'):
            w_conv5 = Weight_variable([3,3,256,512],'w_5')
        with tf.name_scope('biases_5'):
            b_conv5 = biases_variable([512],'b_5')
        with tf.name_scope('conv_5'):
            conv5 = tf.nn.relu(conv2d_P(pool3,w_conv5,1)+b_conv5)

    #conv6#
    with tf.name_scope('layer_6'):
        with tf.name_scope('weight_6'):
            w_conv6 = Weight_variable([3,3,512,512],'w_6')
        with tf.name_scope('biases_6'):
            b_conv6 = biases_variable([512],'b_6')
        with tf.name_scope('conv_6'):
            conv6 = tf.nn.relu(conv2d_P(conv5,w_conv6,1)+b_conv6)
        with tf.name_scope('pool_4'):
            pool4 = max_pooling2x2(conv6)

    #conv7#
    with tf.name_scope('layer_7'):
        with tf.name_scope('weight_7'):
            w_conv7 = Weight_variable([3,3,512,512],'w_7')
        with tf.name_scope('biases_7'):
            b_conv7 = biases_variable([512],'b_7')
        with tf.name_scope('conv_7'):
            conv7 = tf.nn.relu(conv2d_P(pool4,w_conv7,1)+b_conv7)

    #conv8#
    with tf.name_scope('layer_8'):
        with tf.name_scope('weight_8'):
            w_conv8 = Weight_variable([3,3,512,512],'w_8')
        with tf.name_scope('biases_8'):
            b_conv8 = biases_variable([512],'b_8')
        with tf.name_scope('conv_8'):
            conv8 = tf.nn.relu(conv2d_P(conv7,w_conv8,1)+b_conv8)
        with tf.name_scope('pool_5'):
            pool5 = max_pooling2x2(conv8)
    print(pool5.shape)
    #func1#
    with tf.name_scope('layer_9'):
        with tf.name_scope('weight_9'):
            w_conv9 = Weight_variable([3*3*512,4096],'w_9')
        with tf.name_scope('biases_9'):
            b_conv9 = biases_variable([4096],'b_9')
        with tf.name_scope('fc_1'):
            pool5_flat = tf.reshape(pool5,[-1,3*3*512])
            fc1 = tf.nn.relu(tf.matmul(pool5_flat,w_conv9) + b_conv9)
            fc1_drop = tf.nn.dropout(fc1,keep_prob)
    #func2#
    with tf.name_scope('layer_10'):
        with tf.name_scope('weight_10'):
            w_conv10 = Weight_variable([4096,4096],'w_10')
        with tf.name_scope('biases_15'):
            b_conv10 = biases_variable([4096],'b_10')
        with tf.name_scope('fc_2'):
            fc2 = tf.nn.relu(tf.matmul(fc1_drop,w_conv10) + b_conv10)
            fc2_drop = tf.nn.dropout(fc2, keep_prob)
    #func3#
    with tf.name_scope('layer_11'):
        with tf.name_scope('weight_11'):
            w_conv11 = Weight_variable([4096,2],'w_11')
        with tf.name_scope('biases_16'):
            b_conv11 = biases_variable([2],'b_11')
        with tf.name_scope('fc_3'):
            y_pre = tf.nn.softmax(tf.matmul(fc2_drop,w_conv11) + b_conv11)
    with tf.name_scope('cross_entropy'):
        cross_entropy = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_pre), reduction_indices=[1]))
    with tf.name_scope('trian'):
        train_step = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy)
    y_pre_label = tf.argmax(y_pre,1)
    y_label = tf.argmax(y,1)
    a = tf.add(y_pre_label,1)
    b = tf.add(y_label,3)
    c = tf.multiply(a,b)
    tp = tf.reduce_sum(tf.cast(tf.equal(c,8),tf.float32))
    fp = tf.reduce_sum(tf.cast(tf.equal(c,6),tf.float32))
    tn = tf.reduce_sum(tf.cast(tf.equal(c,3),tf.float32))
    fn = tf.reduce_sum(tf.cast(tf.equal(c,4),tf.float32))
    tpr = tp/(tp+fn)
    tnr = tn/(tn+fp)
    tf.summary.scalar('Sensitivity', tpr)
    tf.summary.scalar('Specificity', tnr)
    correct_l1 = tf.equal(y_pre_label,y_label)
    accuracy = tf.reduce_mean(tf.cast(correct_l1,tf.float32))
    tf.summary.scalar('loss',cross_entropy)
    tf.summary.scalar('accuracy',accuracy)
    merged_summary_op = tf.summary.merge_all()
    init = tf.global_variables_initializer()
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    saver = tf.train.Saver({'w_1': w_conv1, 'b_1': b_conv1,
                            'w_2': w_conv2, 'b_2': b_conv2,
                            'w_3': w_conv3, 'b_3': b_conv3,
                            'w_4': w_conv4, 'b_4': b_conv4,
                            'w_5': w_conv5, 'b_5': b_conv5,
                            'w_6': w_conv6, 'b_6': b_conv6,
                            'w_7': w_conv7, 'b_7': b_conv7,
                            'w_8': w_conv8, 'b_8': b_conv8,
                            'w_9': w_conv9, 'b_9': b_conv9,
                            'w_10': w_conv10, 'b_10': b_conv10,
                            'w_11': w_conv11, 'b_11': b_conv11
                            })
    with tf.Session(config=config) as sess:
        sess.run(init)
        writer = tf.summary.FileWriter(saver_path, sess.graph)
        sum_test_acc = 0
        train_x,test_x,train_y,test_y=get_trainsets_and_testsets()#获取训练集和测试集
        num_batch = len(train_x)//batch_size
        for n in range(1,train_time+1):
            sum_train_acc = 0
            av_train_loss = 0
            for i in range(num_batch):#训练
                batch_x = train_x[i*batch_size : (i+1)*batch_size]
                batch_y = train_y[i*batch_size : (i+1)*batch_size]
                _,loss,train_acc= sess.run([train_step,cross_entropy,accuracy],feed_dict={x:batch_x,y:batch_y,keep_prob:dropout_rate})
                sum_train_acc += train_acc
                av_train_loss += loss
            av_train_loss /= num_batch
            sum_train_acc /= num_batch
            test_acc, pre, test_loss = sess.run([accuracy, y_pre, cross_entropy],feed_dict={x: test_x, y: test_y, keep_prob: 1.0})
            if n>30:
                sum_test_acc += test_acc
            if n%10==0:#验证
                print('{0:3d} train_loss: {1:>10.9f} test_loss: {2:>10.9f} train_accuracy:{3:>10.9f} test_accuracy:{4:>10.9f}'.format(n,av_train_loss,test_loss,sum_train_acc,test_acc))
            summary = sess.run(merged_summary_op,feed_dict={x:test_x,y:test_y,keep_prob:1.0})
            writer.add_summary(summary=summary, global_step=n)
        #ROC曲线
        y_,y_p= sess.run([y_label,y_pre],feed_dict={x: test_x, y: test_y, keep_prob: 1.0})
        temp = []
        for i in range(len(y_p)):
            temp.append(y_p[i][1])
        fpr,tpr,thresholds = roc_curve(y_,temp)
        n_auc = auc(fpr,tpr)
        plt.plot(fpr,tpr,linewidth=2)
        plt.plot([0,1],[0,1],'k--')
        plt.axis([0,1,0,1])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        text = 'AUC:'+str(n_auc)[0:5]
        plt.text(0.7,0.2,text,fontdict={'size':'12','color':'b'})
        saver.save(sess,saver_path+'/VGG11')
        print(sum_test_acc/(train_time-30))
        plt.savefig(saver_path+'/VGG11.png')

In [11]:
cnn_train()

(?, 3, 3, 512)
 10 train_loss: 0.632195005 test_loss: 0.531414986 train_accuracy:0.636000003 test_accuracy:0.811320782
 20 train_loss: 0.442552579 test_loss: 0.503996849 train_accuracy:0.811999998 test_accuracy:0.735849082
 30 train_loss: 0.381399847 test_loss: 0.506302953 train_accuracy:0.851999991 test_accuracy:0.754716992
 40 train_loss: 0.351590247 test_loss: 0.496178001 train_accuracy:0.863999991 test_accuracy:0.773584902
 50 train_loss: 0.311850747 test_loss: 0.539997518 train_accuracy:0.883999991 test_accuracy:0.773584902
 60 train_loss: 0.275454279 test_loss: 0.481428951 train_accuracy:0.899999993 test_accuracy:0.792452812
 70 train_loss: 0.235163189 test_loss: 0.504820049 train_accuracy:0.923999991 test_accuracy:0.773584902
 80 train_loss: 0.293890982 test_loss: 0.436840475 train_accuracy:0.887999990 test_accuracy:0.811320782
 90 train_loss: 0.208074013 test_loss: 0.220585197 train_accuracy:0.911999993 test_accuracy:0.924528301
100 train_loss: 0.163698432 test_loss: 0.31334793

AttributeError: 'module' object has no attribute 'to_rgba'